In [14]:
pip install darts

     |████████████████████████████████| 921kB 6.6MB/s eta 0:00:01
     |████████████████████████████████| 317kB 23.9MB/s eta 0:00:01
     |████████████████████████████████| 174kB 19.6MB/s eta 0:00:01
     |████████████████████████████████| 1.2MB 35.8MB/s eta 0:00:01
     |████████████████████████████████| 51kB 12.9MB/s eta 0:00:01
     |████████████████████████████████| 542kB 61.5MB/s eta 0:00:01
     |████████████████████████████████| 1.1MB 56.7MB/s eta 0:00:01
     |████████████████████████████████| 10.8MB 86.1MB/s eta 0:00:01
     |████████████████████████████████| 2.1MB 80.2MB/s eta 0:00:01
     |████████████████████████████████| 819kB 88.3MB/s eta 0:00:01
     |████████████████████████████████| 102kB 23.6MB/s ta 0:00:01
     |█████████████▎                  | 329.4MB 112.4MB/s eta 0:00:05

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



     |████████████████████████▊       | 614.5MB 148.9MB/s eta 0:00:02   | 490.3MB 134.7MB/s eta 0:00:03

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



     |██████████████████████████████▎ | 753.0MB 143.9MB/s eta 0:00:01��████████▋    | 687.8MB 148.9MB/s eta 0:00:01

...

     |████████████████████████████████| 797.1MB 12kB/s 
     |████████████████████████████████| 286kB 71.1MB/s eta 0:00:01
     |████████████████████████████████| 51kB 895kB/s  eta 0:00:01
     |████████████████████████████████| 3.7MB 80.2MB/s eta 0:00:01
     |████████████████████████████████| 204kB 80.9MB/s eta 0:00:01
     |████████████████████████████████| 235kB 98.4MB/s eta 0:00:01
     |████████████████████████████████| 3.6MB 15.0MB/s eta 0:00:01
     |████████████████████████████████| 870kB 80.9MB/s eta 0:00:01
     |████████████████████████████████| 317kB 94.8MB/s eta 0:00:01
     |███████████████▌                | 321.7MB 129.2MB/s eta 0:00:03

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



     |████████████████████████████████| 664.8MB 14kB/s /s eta 0:00:015MB 116.4MB/s eta 0:00:01
     |████████████████████████████████| 121.6MB 59kB/s s eta 0:00:01
     |████████████████████████████████| 829kB 70.9MB/s eta 0:00:01
     |████████████████████████████████| 14.1MB 94.9MB/s eta 0:00:01
     |████████████████████████████████| 23.7MB 74.9MB/s eta 0:00:01
     |████████████████████████████████| 176.3MB 69kB/s /s eta 0:00:01   |██████████                      | 54.9MB 87.7MB/s eta 0:00:0287.7MB/s eta 0:00:02
     |████████████████████████████████| 102kB 21.1MB/s ta 0:00:01
     |████████████████████████████████| 56.5MB 63.3MB/s eta 0:00:01
     |███████████████████████         | 294.0MB 136.4MB/s eta 0:00:01

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



     |████████████████████████████████| 410.6MB 22kB/s 
     |████████████████████████████████| 209.4MB 49kB/s /s eta 0:00:01   |██████████                      | 64.9MB 79.5MB/s eta 0:00:02
     |████████████████████████████████| 1.6MB 43.9MB/s eta 0:00:01
     |████████████████████████████████| 6.2MB 66.7MB/s eta 0:00:01
     |████████████████████████████████| 124.2MB 67kB/s /s eta 0:00:01
     |████████████████████████████████| 196.0MB 72kB/s /s eta 0:00:01  |████████████████▏               | 99.2MB 88.7MB/s eta 0:00:02ta 0:00:01
     |████████████████████████████████| 71kB 3.6MB/s  eta 0:00:01
     |████████████████████████████████| 43.9MB 104.8MB/s eta 0:00:01
     |████████████████████████████████| 542kB 83.0MB/s eta 0:00:01
     |████████████████████████████████| 19.8MB 65.7MB/s eta 0:00:01
     |████████████████████████████████| 143kB 100.4MB/s eta 0:00:01
  Created wheel for pyod: filename=pyod-2.0.2-cp39-none-any.whl size=198482 sha256=98346aeb291500f6a89e8bdbc7a13f867607cc94

In [1]:
import fosforml
from fosforml.model_manager.snowflakesession import get_session
my_session = get_session()

In [2]:
my_session.connection.database

'ASSORTMENT_PLANNING'

In [3]:
my_session.connection.schema

'CPG_BRONZE'

In [4]:
table_name = "ASSORTMENT_PLANNING.CPG_BRONZE.SALES_CLEAN"

In [5]:
sf_df = my_session.sql("select * from {}".format(table_name))
type(sf_df)
df=sf_df.to_pandas()
type(df)

pandas.core.frame.DataFrame

In [6]:
df.head(5)

,MNTH_CODE,TRANS_DATE,START_DATE,SALES_VALUE,SALES_UNITS,SALES_VOLUME,SALES_PTR_VALUE,OC_CODE,DISTRIBUTOR_CODE,OUTLET_CODE,CITY,STATE,COUNTY,STREET,PRODUCT_CODE,CATEGORY,SUBCATEGORY,BRAND
0,202312,2023-12-26,2023-11-27,372.73,2,0.000650,372.727273,202212,DB0110,OL112848,Carbondale,Illinois,City Center,Str2,PRD0118,Soap,Medicated Soap,Burgundy
1,202312,2023-12-26,2023-11-27,436.36,3,0.000900,436.363636,202212,DB0110,OL112848,Carbondale,Illinois,City Center,Str2,PRD0131,Soap,Toilet Soap,Grape vine
2,202312,2023-12-24,2023-11-27,202.82,2,0.000214,209.090909,202212,DB0209,OL112555,Shelbyville,Tennessee,City Center,Str5,PRD0065,Hair Care,Hair Oil,Crimson
3,202312,2023-12-29,2023-11-27,142.86,16,0.000384,142.857143,202212,DB0655,OL113173,Renton,Washington,Orange,Str2,PRD0106,Hair Care,Shampoo,Hair & Shoulders
4,202312,2023-12-17,2023-11-27,145.45,1,0.000300,145.454545,202212,DB0652,OL112373,Wallingford,Connecticut,Dolphin,Str1,PRD0070,Perfume and Deodrants,Female Deodrant,Pista green


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 966400 entries, 0 to 966399
Data columns (total 18 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   MNTH_CODE         966400 non-null  int32  
 1   TRANS_DATE        966400 non-null  object 
 2   START_DATE        966400 non-null  object 
 3   SALES_VALUE       966400 non-null  float64
 4   SALES_UNITS       966400 non-null  int16  
 5   SALES_VOLUME      966400 non-null  float64
 6   SALES_PTR_VALUE   966400 non-null  float64
 7   OC_CODE           966400 non-null  int32  
 8   DISTRIBUTOR_CODE  966400 non-null  object 
 9   OUTLET_CODE       966400 non-null  object 
 10  CITY              966400 non-null  object 
 11  STATE             966400 non-null  object 
 12  COUNTY            966400 non-null  object 
 13  STREET            966400 non-null  object 
 14  PRODUCT_CODE      966400 non-null  object 
 15  CATEGORY          966400 non-null  object 
 16  SUBCATEGORY       96

## Generate Additional Features

In [11]:
import pandas as pd

def generate_features(df):
    # Ensure the TRANS_DATE column is in datetime format
    df['TRANS_DATE'] = pd.to_datetime(df['TRANS_DATE'])
    
    # Extract temporal features
    df['year'] = df['TRANS_DATE'].dt.year
    df['month'] = df['TRANS_DATE'].dt.month
    df['day'] = df['TRANS_DATE'].dt.day
    df['dayofweek'] = df['TRANS_DATE'].dt.dayofweek
    df['quarter'] = df['TRANS_DATE'].dt.quarter
    df['is_month_start'] = df['TRANS_DATE'].dt.is_month_start
    df['is_month_end'] = df['TRANS_DATE'].dt.is_month_end
    
    # Create UNIT_PTR feature
    df['UNIT_PTR'] = df['SALES_PTR_VALUE'] / df['SALES_UNITS']
    
    return df

In [12]:
df = generate_features(df)

In [13]:
df.head()

,MNTH_CODE,TRANS_DATE,START_DATE,SALES_VALUE,SALES_UNITS,SALES_VOLUME,SALES_PTR_VALUE,OC_CODE,DISTRIBUTOR_CODE,OUTLET_CODE,...,SUBCATEGORY,BRAND,year,month,day,dayofweek,quarter,is_month_start,is_month_end,UNIT_PTR
0,202312,2023-12-26,2023-11-27,372.73,2,0.000650,372.727273,202212,DB0110,OL112848,...,Medicated Soap,Burgundy,2023,12,26,1,4,False,False,186.363636
1,202312,2023-12-26,2023-11-27,436.36,3,0.000900,436.363636,202212,DB0110,OL112848,...,Toilet Soap,Grape vine,2023,12,26,1,4,False,False,145.454545
2,202312,2023-12-24,2023-11-27,202.82,2,0.000214,209.090909,202212,DB0209,OL112555,...,Hair Oil,Crimson,2023,12,24,6,4,False,False,104.545455
3,202312,2023-12-29,2023-11-27,142.86,16,0.000384,142.857143,202212,DB0655,OL113173,...,Shampoo,Hair & Shoulders,2023,12,29,4,4,False,False,8.928571
4,202312,2023-12-17,2023-11-27,145.45,1,0.000300,145.454545,202212,DB0652,OL112373,...,Female Deodrant,Pista green,2023,12,17,6,4,False,False,145.454545


In [20]:
df = df.sort_values(by='TRANS_DATE')

In [23]:
# Convert all column names to uppercase
df.columns = [col.upper() for col in df.columns]

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 966400 entries, 787515 to 905006
Data columns (total 26 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   MNTH_CODE         966400 non-null  int32         
 1   TRANS_DATE        966400 non-null  datetime64[ns]
 2   START_DATE        966400 non-null  object        
 3   SALES_VALUE       966400 non-null  float64       
 4   SALES_UNITS       966400 non-null  int16         
 5   SALES_VOLUME      966400 non-null  float64       
 6   SALES_PTR_VALUE   966400 non-null  float64       
 7   OC_CODE           966400 non-null  int32         
 8   DISTRIBUTOR_CODE  966400 non-null  object        
 9   OUTLET_CODE       966400 non-null  object        
 10  CITY              966400 non-null  object        
 11  STATE             966400 non-null  object        
 12  COUNTY            966400 non-null  object        
 13  STREET            966400 non-null  object        
 14  PROD